# Imports
do not edit

In [1]:
import pandas as pd
pd.set_option('display.max_columns', 500)

# Change Me

In [2]:

name1 = "SMR" # Temp 1 abbreviation
name2 = "YZC" # Temp 2 abbreviation
namecol = "SID1" # Name of the column for staff ID
token = "token" # spelling of "token" column. Capitalization matters.

# if reading excel files, change the "csv" to "excel" and change the filename. Take note it will only read the first sheet.
data = pd.read_csv("data/comb2.csv")

# test cases for testing code
#test1 = pd.read_csv("test/test1.csv") # control
#test2 = pd.read_csv("test/test2.csv") # differences in values
#test3 = pd.read_csv("test/test3.csv") # differences in rows


# Function
do not edit

In [4]:
data1 = data[data[namecol]==name1]
data2 = data[data[namecol]==name2]

#function to extract missing rows and equalize datasets

def de_dupe_clean(df1, df2, namecol, token):

    # removing name columns (if datasets start off merged, and are differentiated by a column's value)
    
    df1 = df1.drop(columns = namecol)
    df2 = df2.drop(columns = namecol)
    
    # merging datasets to extract missing rows
    
    df_merged = pd.merge(df1, 
                         df2,
                         how = 'outer',
                         left_on = [token],
                         right_on = [token], 
                         indicator = True)

    # extract missing rows from df1
    
    df1_missing = df_merged[df_merged["_merge"] == "left_only"]
    df1_mis_token = list(df1_missing[token])
    
    # extract missing rows from df2
    
    df2_missing = df_merged[df_merged["_merge"] == "right_only"]
    df2_mis_token = list(df2_missing[token])
    
    # removing missing rows from both df1 and df2
    
    df1_clean = df1[~df1[token].isin(df1_mis_token)].sort_values(by = token)
    df2_clean = df2[~df2[token].isin(df2_mis_token)].sort_values(by = token)
    
    df1_clean.reset_index(drop = True, inplace = True)
    df2_clean.reset_index(drop = True, inplace = True)
    
    # comparing datasets
    
    changes = df1_clean.compare(df2_clean)
    
    # extracting tokens for rows with differences
    
    changes[token] = [df1_clean[token].iloc[i] for i in changes.index]
    changes.set_index(token, inplace = True)
    
    return df1_mis_token, df2_mis_token, changes

# function to print different results based on whether there are changes

def de_dupe_check(df1, df2, namecol, token):

    mis1, mis2, change = de_dupe_clean(df1, df2, namecol, token)

    # printing missing rows
    
    if len(mis1) > 0:
        print (f"The following {len(mis1)} tokens are missing from {name2}'s dataset: {mis1}")
    if len(mis2) > 0:
        print (f"The following {len(mis2)} tokens are missing from {name1}'s dataset: {mis2}")
    if len(mis1) == 0 and len(mis2) == 0:
        print ("number of rows are identical")

    # printing rows with differences
        
    print ("\n----------\n")
    if len(change) == 0 and len(mis1) == 0 and len(mis2) == 0:
        print ("datasets are identical")
    elif len(change) == 0:
        print ("other than missing rows, datasets are identical")
    else:
        results = pd.DataFrame(data = change)
        results.to_excel("output/output.xlsx")
        
de_dupe_check(data1, data2, namecol = namecol, token = token)

The following 44 tokens are missing from YZC's dataset: ['2090574M', '3037490M', '6075068M', '2080789M', '2023468M', '1042834M', '5068494M', '1078035M', '1070886M', '7079115M', '2078062M', '5044397M', '9060268M', '7074125M', '5011962M', '4064830M', '6066041M', '9089513M', '1056421M', '7075990M', '6089504M', '2105686M', '7039396M', '9003223M', '2012064M', '3099308M', '3033710M', '4091856M', '8048185M', '7030225M', '4043360M', '9081160M', '2049574M', '2068464M', '1048339M', '7044434M', '4001708M', '9029392M', '8000553M', '3044606M', '5076102M', '4055376M', '1039181M', '2019513M']

----------



# Check the output folder for the excel file